[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/adams/blob/master/exercises/tut5_embeddings_teacher.ipynb)

# Tutorial 5: Word embeddings and Word2Vec
The tutorial covers word embeddings in general and one of the most well-known models in this matter, the so-called Word-to-Vec (W2V). For this purpose, we reconsider the text dataset IMDB from the last tutorial. Yet, this time we preprocess the data with `Keras` using the `TextVectorization` layer that facilitates the standardization, tokenization and indexing. Next, we create a simple binary classification model using word embeddings to grasp the essence in practice. Finally, we apply the W2V model to the same data using [Gensim library](https://pypi.org/project/gensim/). 

Several libraries make things easier if the aim is to use W2V directly. The [Gensim library](https://pypi.org/project/gensim/) is one of them that offers a friendly interface to train embeddings, as you will see in this tutorial. 

However, if you would like to start from scratch and code W2V yourself using just `NumPy`, we recommend [Nathan Rooy's post](https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/). Or, if you would like to do it with `TensorFlow`, there is an excellent tutorial [here](https://www.tensorflow.org/tutorials/text/word2vec) from the TensorFlow website.

The outline then is the following
1. Preparing the IMDB dataset with `Keras`
2. Understanding embeddings with a simple binary classification model
3. Word2Vec using `Gensim`

For further examples, please visit the demo [word-2-vec.ipynb](https://github.com/Humboldt-WI/adams/blob/master/demos/nlp/word-2-vec.ipynb).

## 1. Preparing the IMDB dataset
Setting things up

In [2]:
# Import the required libraries
import io
import re
import string
import pandas as pd
import gensim
from gensim.models import Word2Vec  
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Exercise 1
Load the `IMDB-50K-Movie-Review.zip` file, and map the labels to 1 (positive) and 0 (negative). Then, have a look at the first rows.

In [98]:
# load the data (be sure to provide the correct file path)
total_imbd = pd.read_csv("IMDB-50K-Movie-Review.zip", sep=",", encoding="ISO-8859-1")
total_imbd['sentiment'] = total_imbd['sentiment'].map({'positive' : 1, 'negative': 0})
print(total_imbd.head())
print(total_imbd.shape)

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1
(50000, 2)


### Exercise 2
Split the data into training and validation sets. Use 80% of the data for training. You can use `train_test_split()` function from `sklearn`. In addition, transform the sets into `NumPy` arrays using `to_numpy()`.

In [99]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(total_imbd['review'], total_imbd['sentiment'], test_size = 0.2, random_state = 5)
# transform them to numpy
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

So far, we have just created training and validation sets of text and labels. However, we cannot feed a neural network with this text format. We need numeric tensors. 

The transformation of text to numeric tensors is known as *vectorization*. This process can be split into three steps:
1. **Standardization** of the text, such as removing punctuation, converting all the text to lowercase, etc.
2. **Tokeinzation** of the standardized text, where we separate the text into units or *tokens*, usually words or n-grams.
3. **Indexing** of the tokens into a numerical vector.

These 3 steps are implemented in the Keras `TextVectorization` layer.

```python
TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )
```
Where `our_standardization` is our customized function to standardize the text, for example, we saw that some examples in the dataset have HTML tags `<br />`, and we'd like to delete them. 

### Exercise 3
So, let's first build our own standardization function called `our_standardization`. The function should convert uppercase to lowercase (`tf.strings.lower`), remove HTML tags (`tf.strings.regex_replace`), deletes the punctuation (`re.escape(string.punctuation)`) and double spaces.

In [103]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase

  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags, alternatively remove html characters
  #in advance with beautifulsoup.
  
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
    
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [100]:
re.escape(string.punctuation)

'!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~'

In [101]:
'[%s]' % re.escape(string.punctuation)

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [102]:
'[%s]' % 'example'

'[example]'

### Exercise 4
Apply the `our_standardization` function to the following text and see how it works

`"Bruce Dern also is in the mix and Dern never fails to fascinate in about any film.<br /><br />The movie could be considered kind of downer to the average viewer, but I found it fascinating....and I don't like depressing movies normally. What I found was a kind of quirky crime film."`

In [104]:
# An example of the our_standardization function
our_standardization("Bruce Dern also is in the mix and Dern never fails to fascinate in about any film.<br /><br />The movie could be considered kind of downer to the average viewer, but I found it fascinating....and I don't like depressing movies normally. What I found was a kind of quirky crime film.")

<tf.Tensor: shape=(), dtype=string, numpy=b'bruce dern also is in the mix and dern never fails to fascinate in about any film the movie could be considered kind of downer to the average viewer but i found it fascinatingand i dont like depressing movies normally what i found was a kind of quirky crime film'>

### Exercise 5
Great! So let's now vectorize our data (use `TextVectorization`) with a vocabulary of the first 10,000 most frequent words and a maximum sequence of the text of 100 characters. Called this layer `vectorize_layer`.

In [105]:
# Define the size of the vocabulary and the max number of words in a sequence
vocab_size = 10000
seq_length = 100

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )


### Exercise 6
Index the vocabulary. To do it, you need to call the `adapt()` method from the `vectorize_layer` and apply it to `X_train`. Then, retrieve the computed vocabulary using `get_vocabulary()` and save it into `vocab`. Finally, print the first 10 words of the vocabulary (`print(vocab[:10])`).

In [106]:
# To create the vocabulary, we need to call adapt. The input is only the text
vectorize_layer.adapt(X_train)
# Check the first 10 words of the vocabulary. It is sorted by frequency 
vocab = vectorize_layer.get_vocabulary()
print(vocab[:10])

['', '[UNK]', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'it']


### Exercise 7
Apply the `vectorization_layer` to the same previous example, i.e. To

`"Bruce Dern also is in the mix and Dern never fails to fascinate in about any film.<br /><br />The movie could be considered kind of downer to the average viewer, but I found it fascinating....and I don't like depressing movies normally. What I found was a kind of quirky crime film."`


In [107]:
# Check the vectorization layer
vectorize_layer(["Bruce Dern also is in the mix and Dern never fails to fascinate in about any film.<br /><br />The movie could be considered kind of downer to the average viewer, but I found it fascinating....and I don't like depressing movies normally. What I found was a kind of quirky crime film."])

<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[1389, 8562,   82,    7,    8,    2, 1579,    4, 8562,  109,  976,
           6,    1,    8,   42,   99,   19,    2,   17,   97,   26, 1206,
         236,    5,    1,    6,    2,  871,  527,   18,   10,  245,    9,
           1,   10,   89,   38, 2234,   92, 1805,   48,   10,  245,   13,
           3,  236,    5, 2653,  832,   19,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]], dtype=int64)>

To better understand what is happening in the TextVectorization layer we can use less data, smaller vocab_size and smaller number of max_tokens

In [108]:
# Create a vectorization layer
vectorize_layer_smaller = TextVectorization(
    standardize = our_standardization,
    max_tokens = 10,
    output_sequence_length = 20
    )

In [109]:
import numpy as np
X_tr=np.array([["Bruce Dern also is in the mix and Dern never fails to fascinate in about"],
              ["Bruce Dern also "],
              ["Bruce Dern also is in the mix"],
              ["Bruce Dern also is in the mixdasfdsfds"]])



In [110]:
vectorize_layer_smaller.adapt(X_tr)
# Check the first 10 words of the vocabulary. It is sorted by frequency 
vocab_smaller = vectorize_layer_smaller.get_vocabulary()
vocab_smaller

['', '[UNK]', 'dern', 'in', 'bruce', 'also', 'the', 'is', 'mix', 'to']

In [111]:
X_tr_vec=vectorize_layer_smaller(X_tr)

In [112]:
X_tr_vec

<tf.Tensor: shape=(4, 20), dtype=int64, numpy=
array([[4, 2, 5, 7, 3, 6, 8, 1, 2, 1, 1, 9, 1, 3, 1, 0, 0, 0, 0, 0],
       [4, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 2, 5, 7, 3, 6, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 2, 5, 7, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)>

In [113]:
X_tr_words = [[vocab_smaller[w] for w in rev] for rev in X_tr_vec]
print(X_tr_words[0],'\n',X_tr_words[1],'\n',X_tr_words[2],'\n',X_tr_words[3])

['bruce', 'dern', 'also', 'is', 'in', 'the', 'mix', '[UNK]', 'dern', '[UNK]', '[UNK]', 'to', '[UNK]', 'in', '[UNK]', '', '', '', '', ''] 
 ['bruce', 'dern', 'also', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''] 
 ['bruce', 'dern', 'also', 'is', 'in', 'the', 'mix', '', '', '', '', '', '', '', '', '', '', '', '', ''] 
 ['bruce', 'dern', 'also', 'is', 'in', 'the', '[UNK]', '', '', '', '', '', '', '', '', '', '', '', '', '']


## 2. Understanding word embeddings
We have not introduced any embedding layer until now; instead, we created a vectorization layer that can transform text inputs into numeric tensors. So, for example, the text "tomorrow is Saturday" will be transformed into something like `[23, 45, 5, 0, 0, 0]` (if the length of the sequence is 6). An embedding layer transforms each word, which can be thought of as a one-hot vector, into another more dense vector. 

Let's see an example of the `Embedding` layer in Keras, where we hypothetically have only 100 words in the vocabulary, and we want to transform this space into a 5-dimensional one.

In [114]:
# Create the embedding layer of shape (100,5)
embedding_layer = layers.Embedding(100, 5)
# Feed a sequence of word indices
result1 = embedding_layer(tf.constant([23, 45, 5, 0, 0, 0]))
# We can also feed batches    
result2 = embedding_layer(tf.constant([[23, 45, 5, 0, 0, 0],
                                       [3, 4, 55, 4, 0, 0]]))
print("result1:",result1.shape,"\nresult2:",result2.shape)

result1: (6, 5) 
result2: (2, 6, 5)


Each word index has been transformed into a 5-dimensional vector (in this case, random values). For example, the values of `results1` are

In [115]:
result1

<tf.Tensor: shape=(6, 5), dtype=float32, numpy=
array([[ 0.00982534, -0.01375347,  0.04213485,  0.01328217,  0.00841175],
       [ 0.0378991 , -0.00308871,  0.03644396, -0.03695422,  0.0240776 ],
       [-0.02275083, -0.04164793, -0.04117155,  0.02198904, -0.04802036],
       [-0.02885591,  0.01358051, -0.01547223,  0.0012092 ,  0.01538805],
       [-0.02885591,  0.01358051, -0.01547223,  0.0012092 ,  0.01538805],
       [-0.02885591,  0.01358051, -0.01547223,  0.0012092 ,  0.01538805]],
      dtype=float32)>

In [116]:
embedding_layer.trainable_variables[0][23]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.00982534, -0.01375347,  0.04213485,  0.01328217,  0.00841175],
      dtype=float32)>

In [117]:
embedding_layer.trainable_variables[0][45]

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.0378991 , -0.00308871,  0.03644396, -0.03695422,  0.0240776 ],
      dtype=float32)>

The embedding layer: dictionary which maps integer indices of inputs to dense vectors (embedding weights). 

### Exercise 8
Build a simple model to infer the sentiment. To do it, use a `Sequential` model where the first layer transforms the text into tensors (`vectorize_layer`), the second layer embeds the vocabulary into a 16-dimension (`layers.Embedding`), the third layer uses `layers.GlobalAveragePooling1D()` to reduce the complete text to a single average vector in the embedding space, and finally, use a `Dense` layer with a sigmoid activation to infer the sentiment.

In [118]:
# Create a simple model to use word embeddings
embedding_dim = 16
model = Sequential([
  vectorize_layer,
  layers.Embedding(vocab_size, embedding_dim, name="embedding"), 
  layers.GlobalAveragePooling1D(), # each sample is reduced to the average of the word embeddings
  layers.Dense(1, activation='sigmoid')
])


### Exercise 9 
Compile the model with the `rmsprop` optimizer, the adequate loss function and monitor the `accuracy`.

In [119]:
# let's compile it
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', # positive or negative
              metrics=['accuracy'])

### Exercise 10
Fit the model using 10 `epochs`. Remember to specify the validation dataset in `validation_data`. How accurate is the model in the validation set? How many parameters does it have?

In [120]:
# ~ 2 minutes
model.fit(
    X_train,
    y_train,
    validation_data = (X_val, y_val),
    epochs=10)

Epoch 1/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.6550 - accuracy: 0.6613 - val_loss: 0.5998 - val_accuracy: 0.7183
Epoch 2/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.5379 - accuracy: 0.7536 - val_loss: 0.4934 - val_accuracy: 0.7716
Epoch 3/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.4537 - accuracy: 0.7939 - val_loss: 0.4390 - val_accuracy: 0.7979
Epoch 4/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.4075 - accuracy: 0.8174 - val_loss: 0.4104 - val_accuracy: 0.8140
Epoch 5/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.3787 - accuracy: 0.8331 - val_loss: 0.3935 - val_accuracy: 0.8198
Epoch 6/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.3586 - accuracy: 0.8433 - val_loss: 0.3824 - val_accuracy: 0.8267
Epoch 7/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.3438 - accuracy: 0.8504 - val_loss: 0.3763 - val_accuracy:

In [121]:
# Check the number of trainable parameters
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,017
Trainable params: 160,017
Non-trainable params: 0
_________________________________________________________________


In [136]:
for lr in model.layers:
    print('Layer: ',lr)
    nr_tr_vars=len(lr.trainable_variables)
    print('Output shape: ',lr.output_shape)
    print('Number of weight matrices of trainable variables: ',nr_tr_vars)
    if nr_tr_vars>0:
        for nr_tr in range(0,nr_tr_vars):
            print('Trainable Weight matrix ',nr_tr+1,': ',lr.trainable_variables[nr_tr].shape)
    print('\n')

Layer:  <keras.layers.preprocessing.text_vectorization.TextVectorization object at 0x00000213896955B0>
Output shape:  (None, 100)
Number of weight matrices of trainable variables:  0


Layer:  <keras.layers.core.embedding.Embedding object at 0x000002139175FEE0>
Output shape:  (None, 100, 16)
Number of weight matrices of trainable variables:  1
Trainable Weight matrix  1 :  (10000, 16)


Layer:  <keras.layers.pooling.global_average_pooling1d.GlobalAveragePooling1D object at 0x000002139175FF40>
Output shape:  (None, 16)
Number of weight matrices of trainable variables:  0


Layer:  <keras.layers.core.dense.Dense object at 0x0000021391838370>
Output shape:  (None, 1)
Number of weight matrices of trainable variables:  2
Trainable Weight matrix  1 :  (16, 1)
Trainable Weight matrix  2 :  (1,)




## Word2Vec with Gensim library
Let's now apply W2V to the same text data. Remember that W2V proposes two models for learning word vectors, continuous-bag-of-words (CBOW) and Skip-Gram (SG). In a nutshell, CBOW predicts a central <font color='yellow'>target</font> word from surrounding <font color='green'>context</font> words, while SG takes the opposite approach. Given a <font color='yellow'>target</font> word, predict <font color='green'>context</font> words. For example, using a window size of 2 to the following phrase

> I finally <font color='green'>found a</font><font color='yellow'> machine</font><font color='green'> at the </font>  gym that I like: the vending machine!

So in CBOW, the problem is 

[I finally <font color='green'>found a</font><font color='yellow'> ?</font><font color='green'> at the </font> gym]

And in SG,

[I finally <font color='green'>? ?</font><font color='yellow'> machine</font><font color='green'> ? ? </font> gym]

In this tutorial, we apply SG (argument `sg=1` in `gensim`). But you are welcome to compare results for CBOW (`sg=0`).

### Exercise 11
Create `X_train_vec` by applying the vectorization layer you have already created to `X_train`. In this way, we will be using the same vectorization procedure as before (same vocabulary, length of the sequence, etc.)

In [137]:
# Apply vectorize_layer to X_train
X_train_vec = vectorize_layer(X_train)


In [138]:
X_train_vec[0]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([  46,    7,   33,  312,  278, 7117,  403,  789,    6,  388,    9,
         57,   25, 7288,    8, 2121,   15,    3,    1,    1,    2,   17,
       4666, 1092,   98,  202,    4,   13,    1,  339,   40,    3, 3421,
        734, 9110, 1176,    6, 3517,    4, 5112,   36,  514,   67,    3,
       3593,  106, 2160,  155,   63,  355,   94,    2,   80,  410,   23,
         14, 1285,    4,  221,   14,  777, 2864,   14, 3624,    1,  110,
         10,  742,    6, 4417,    2,  219,   18,    2, 5407,   27, 2612,
         12, 1653,    7,   24,    1,    2,  123,    2,   17,  208,   31,
          1,  649,   10,   38,    2,  187,   12,    1,  594, 2654,   13,
        343], dtype=int64)>

### Exercise 12
Gensim accepts words, so convert the `X_train_vec` into a list of words. Called this object `X_train_words`

In [139]:
# ~ 6 min
X_train_words = [[vocab[w] for w in rev] for rev in X_train_vec]

### Exercise 13
Train a W2V model using `Word2Vec` function and `X_train_words` as input (call it `w2v_model`). Use `min_count` of 1, a `window` of 5, 50 `epochs`, a `vector_size` of 100 for the embeddings and SG (`sg=1`).

In [140]:
# Train a Word2Vec model ~ 5 min
w2v_model = Word2Vec(X_train_words, #X_train_vec.numpy(), 
                 min_count=1,  #min_count means the frequency benchmark
                 window=5,     #the size of context
                 epochs=50,  
                 vector_size=100, #size of embedding
                 workers=4,#for parallel computing
                 sg  = 1)    
# summarize the loaded model
print(w2v_model)


Word2Vec(vocab=10000, vector_size=100, alpha=0.025)


In [4]:
help(Word2Vec)

Help on class Word2Vec in module gensim.models.word2vec:

class Word2Vec(gensim.utils.SaveLoad)
 |  Word2Vec(sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), comment=None, max_final_vocab=None, shrink_windows=True)
 |  
 |  Method resolution order:
 |      Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=100

In [149]:
w2v_model.wv['great']

array([-0.7422866 ,  0.07011281,  0.17215607, -0.06441133, -0.18116634,
       -0.3497524 , -0.1832129 ,  0.19918127,  0.05572874, -0.35483485,
        0.0619622 , -0.19628175,  0.24171843, -0.23548499,  0.28379336,
        0.21866822, -0.00950472, -0.08436187,  0.23246624, -0.37449208,
        0.02789638,  0.06735098,  0.30410182,  0.15329996, -0.24441405,
        0.2002234 , -0.01896348,  0.1310914 , -0.08164166,  0.2830613 ,
       -0.07862464, -0.11080905, -0.01053677,  0.17467418,  0.01008057,
        0.20077197,  0.14305808, -0.06565698, -0.30597922,  0.13186269,
        0.1908682 , -0.02755645, -0.05216184,  0.3922729 , -0.09934577,
        0.21803425, -0.06046042, -0.19633771, -0.13130325,  0.37006807,
       -0.1257119 , -0.19493343,  0.17493409, -0.20511326, -0.13973004,
        0.30518663, -0.15295902, -0.12946807,  0.22887555,  0.3490611 ,
       -0.13998403,  0.08421545,  0.20027298,  0.2395708 , -0.08650368,
        0.10091771,  0.21836822,  0.1443163 , -0.11423711,  0.15

Words below the min_count frequency are dropped before training occurs.

### Exercise 14
Check how similar are great to good, great to horrible and so on. Use `w2v_model.wv.similarity()` function.

In [142]:
print(w2v_model.wv.similarity('great', 'good'))
print(w2v_model.wv.similarity('great', 'horrible'))

0.71228135
0.3995029


### Exercise 15
Get the `topn` 5 most similar words to great. Use `w2v_model.wv.most_similar` function

In [59]:
w2v_model.wv.most_similar('great', topn = 5)

[('wonderful', 0.8117287158966064),
 ('fantastic', 0.7801386117935181),
 ('fine', 0.7010308504104614),
 ('good', 0.6951696872711182),
 ('terrific', 0.6847767233848572)]